In [ ]:
import sqlite3 as sql
from contextlib import closing
import pandas as pd

In [ ]:
queryInterConn = "SELECT pointLabel,pointTpMapX,pointTpMapY,pointKey,fromBzLabelLong FROM Interconnections"
queryBalanZone = "SELECT bzLabel,tpMapX,tpMapY,id,bzTooltip from balancingzones"

with closing(sql.connect('entsog.db')) as conn:
    interconnections = pd.read_sql_query(queryInterConn,conn)
    balancingZones = pd.read_sql_query(queryBalanZone,conn)

In [ ]:
scale=0.2
balancingZones['long']=balancingZones['tpMapX']*scale+15
balancingZones['lat']=balancingZones['tpMapY']*scale+50
import plotly.express as px
fig = px.scatter_mapbox(balancingZones, lat="lat", lon="long", hover_name="bzLabel", hover_data=["bzTooltip", "id"],
                        color_discrete_sequence=["fuchsia"], zoom=2, height=400)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
scale=0.2
interconnections['long']=interconnections['pointTpMapX']*scale+15
interconnections['lat']=interconnections['pointTpMapY']*scale+50

import plotly.express as px
fig = px.scatter_mapbox(interconnections[:], lat="lat", lon="long", hover_name="pointLabel", hover_data=["pointKey", "fromBzLabelLong"],
                        color_discrete_sequence=["fuchsia"], zoom=2, height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
import plotly.graph_objects as go

# Create figure
'''
fig = go.Figure()

# Add trace
fig.add_trace(
    #go.Scatter(x=[0, 0.5, 1, 2, 2.2], y=[1.23, 2.5, 0.42, 3, 1])
    go.Scatter(x=balancingZones['tpMapX']+350,y=balancingZones['tpMapY']+150,mode='markers')
               #hover_name="bzLabel", hover_data=["bzTooltip", "id"])    
)
'''

def plotData(fig,scale_factor):
    from PIL import Image; 
    img = Image.open('europe.jpg')
    img_width = 2000
    img_height = 1644

    # Configure axes
    fig.update_xaxes(
        visible=True,
        range=[0, img_width * scale_factor]
    )

    fig.update_yaxes(
        visible=True,
        range=[0, img_height * scale_factor],
        # the scaleanchor attribute ensures that the aspect ratio stays constant
        scaleanchor="x"
    )

    # Add image
    fig.add_layout_image(
        dict(
            x=100,
            sizex=img_width * scale_factor,
            y=img_height * scale_factor,
            sizey=img_height * scale_factor,
            xref="x",
            yref="y",
            opacity=0.5,
            layer="below",
            #sizing="stretch",
            source=img)
    )

    # Configure other layout
    fig.update_layout(
        width=img_width * scale_factor,
        height=img_height * scale_factor,
        margin={"l": 0, "r": 0, "t": 0, "b": 0},
    )

    # Disable the autosize on double click because it adds unwanted margins around the image
    # More detail: https://plotly.com/python/configuration-options/
    fig.show(config={'doubleClick': 'reset'})

In [ ]:

scale_factor = 0.5

interconnections['xs']=(interconnections['pointTpMapX']*5+1200)*scale_factor
interconnections['ys']=(interconnections['pointTpMapY']*5+600)*scale_factor

fig = px.scatter(interconnections,x="xs",y="ys",hover_name="pointLabel", hover_data=["pointKey", "fromBzLabelLong"])
plotData(fig,scale_factor)

In [ ]:

scale_factor = 0.5

balancingZones['xs']=(balancingZones['tpMapX']*5+1200)*scale_factor
balancingZones['ys']=(balancingZones['tpMapY']*5+600)*scale_factor

fig = px.scatter(balancingZones,x="xs",y="ys",hover_name="bzLabel", hover_data=["bzTooltip", "id"])
plotData(fig,scale_factor)

# Data-Widget

In [ ]:
# visualize Data
def getData(name,direction=None):
    conn = sql.connect('entsog.db')
    query = "SELECT day,month,year,sum(value) as value FROM AggregatedData WHERE directionKey='"+direction+"' and adjacentSystemsLabel='"+name+"' group by day,month,year"

    df = pd.read_sql_query(query,conn)
    df['begin']=pd.to_datetime(df[['day','month','year']])
    df.index = df['begin']
    df.sort_index(inplace=True)
    return df
    
#%time data2 = getData('NCG','entry')
def update(name):
    entrys = getData(name,'entry')
    exits = getData(name,'exit')
    diff = entrys['value']-exits['value']
    entrys['diff']= diff
    entrys['exit']=exits['value']
    
    return entrys

#%time data2 = update('NCG')
#data2.sort_index(inplace=True)

In [ ]:
def getData(name,direction=None):
    conn = sql.connect('entsog.db')

    if direction=='entry':
        query = "SELECT periodFrom,value FROM AggregatedData WHERE directionKey='entry' and adjacentSystemsLabel='"+name+"'"
    elif direction=='exit':
        query = "SELECT periodFrom,value FROM AggregatedData WHERE directionKey='exit' and adjacentSystemsLabel='"+name+"'"
    else:
        query = "select value-exit_value as diff,a.periodFrom,value, exit_value from (SELECT periodFrom,value FROM AggregatedData WHERE directionKey='entry' and adjacentSystemsLabel='"+name+"') a"
        query += " join (select periodFrom,value as exit_value from AggregatedData where directionKey='exit' and adjacentSystemsLabel='"+name+"') b on a.periodFrom = b.periodFrom"
        #query += " where a.periodFrom < '2018-01-01'"
        #and operatorKey='BE-TSO-0001'

    df = pd.read_sql_query(query,conn)
    df['begin']=pd.to_datetime(df['periodFrom']).dt.to_period('M').dt.to_timestamp()
    df['begin']=pd.to_datetime(df['periodFrom']).dt.floor('d')

    sums = df.groupby('begin').sum()
    sums['begin']=sums.index
    sums.sort_index(inplace=True)
    return sums

def update(name):
    entrys = getData(name,'entry')
    exits = getData(name,'exit')
    diff = entrys['value']-exits['value']
    entrys['diff']= diff
    entrys['exit']=exits['value']
    return entrys

#%time data = getData('NCG','entry')
%time data = update('NCG')
    

In [ ]:
#operatorKey
with closing(sql.connect('entsog.db')) as conn:
    operators2 = pd.read_sql_query('select distinct operatorKey,operatorLabel from AggregatedData',conn)
    operators = pd.read_sql_query('select distinct adjacentSystemsKey,adjacentSystemsLabel from AggregatedData',conn)
    
operators['operators']=operators['adjacentSystemsLabel']

anforderungen:
Dataset middt Monat, Tag, Differenz (entry,exit), operatorKey,adjacentSystemsLabel

In [ ]:
import plotly.graph_objects as go
from ipywidgets import widgets

month = widgets.IntSlider(
    value=1.0,
    min=1.0,
    max=12.0,
    step=1.0,
    description='Month:',
    continuous_update=False
)

use_date = widgets.Checkbox(
    description='Date: ',
    value=True,
)

container = widgets.HBox(children=[use_date, month])

textbox = widgets.Dropdown(
    description='Operators:   ',
    value=operators['operators'][0],
    options=operators['operators'].tolist()
)


trace1 = go.Scatter(x=data['begin'],y=data['value'],name='importiertes Gas')#,mode='markers')
trace2 = go.Scatter(x=data['begin'],y=data['exit'],name='exportiertes Gas')#,mode='markers')
trace3 = go.Scatter(x=data['begin'],y=data['diff'],name='dem Gebiet beigefügtes Gas')#,mode='markers')
               #hover_name="bzLabel", hover_data=["bzTooltip", "id"])    
# Assign an empty figure widget with two traces
#trace1 = go.Histogram(x=df['arr_delay'], opacity=0.75, name='Arrival Delays')
#trace2 = go.Histogram(x=df['dep_delay'], opacity=0.75, name='Departure Delays')
g = go.FigureWidget(data=[trace1, trace2,trace3],
                    layout=go.Layout(
                        title=dict(
                            text='ENTSOG Physical Flow'
                        ),
                        xaxis=dict(title='Datum'),
                        yaxis=dict(title='gefördertes Gas in kWh')
                        #barmode='overlay'
                    ))

In [ ]:
def validate():
    if textbox.value in operators['operators'].to_list():
        return True
    else:
        return False


def response(change):
    if validate():
        print('updating...')
        temp_df = update(textbox.value)
        x1 = temp_df['value']
        x2 = temp_df['exit']
        x3 = temp_df['diff']
        print('finished updating')
        with g.batch_update():
            g.data[0].y = x1
            g.data[1].y = x2
            g.data[2].y = x3
            #g.layout.xaxis.title = 'Datum'
            #g.layout.yaxis.title = 'gefördertes Gas in kWh'
    else:
        print('Invalid name: ', textbox.value)

textbox.observe(response, names="value")
#month.observe(response, names="value")
#use_date.observe(response, names="value")


In [ ]:
container2 = widgets.HBox([textbox])
widgets.VBox([container2,
              g])